## Course: TM10007 - Machine learning
Editors: Lishia Vergeer, Amy Roos, Maaike Pruijt, Hilde Roording.

Description: The aim of this code is to predict the tumor grade of glioma’s(high or low) before surgery, 
based on features extracted from a combination of four MRI images: 
T2-weighted, T2-weighted FLAIR and T1-weighted before and after injection of contrast agent.

#### Import packages

In [15]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
# from sklearn import decomposition
# import seaborn

# Import code
from brats.load_data import load_data
from sklearn.model_selection import train_test_split

# scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Classifiers
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm

#### Load data

In [5]:
# Data loading functions. Uncomment the one you want to use
from brats.load_data import load_data

X = load_data()
print(f'The number of samples in X: {len(X.index)}')
print(f'The number of columns in X: {len(X.columns)}')


The number of samples in X: 167
The number of columns in X: 725


c:\Users\maaik\Desktop\TM10007 Machine learning\TM10007_ML\TM10007\brats\load_data.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data2)


#### Split data in X and y
Split in X (data) and y (label)


- Checken of splitten van data en labels inderdaad moet na het er uithalen van NaN's --> Nu ervoor!

In [6]:
# split column label from dataset X
y = X['label']
print(f'The number of samples in y: {len(y.index)}')

# print dataframe X (features)
df = pd.DataFrame(X)
print(df.head())
# print dataframe y (labels)
df2 = pd.DataFrame(y)
print(df2)

The number of samples in y: 167
              VOLUME_ET  VOLUME_NET  VOLUME_ED  VOLUME_TC  VOLUME_WT  \
ID                                                                     
TCGA-02-0006       1662         384      36268       2046      38314   
TCGA-02-0009       4362        4349      15723       8711      24434   
TCGA-02-0011      33404       48612      45798      82016     127814   
TCGA-02-0027      12114        7587      34086      19701      53787   
TCGA-02-0033      34538        7137      65653      41675     107328   

              VOLUME_BRAIN  VOLUME_ET_OVER_NET VOLUME_ET_OVER_ED  \
ID                                                                 
TCGA-02-0006       1469432            4.328125          0.045826   
TCGA-02-0009       1295721            1.002989          0.277428   
TCGA-02-0011       1425843            0.687155          0.729377   
TCGA-02-0027       1403429            1.596679          0.355395   
TCGA-02-0033       1365237            4.839288         

#### Split data
This function creates a panda dataframe and splits the data into test and train components.
This is done with test_size variable and the function train_test_split from the sklearn module.
Returns a train set with the data of 80% and a test set of 20% of the subjects.



In [14]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)  

# print(f'data_train: {X_train}')
# print(f'data_test: {X_test}')

In [8]:
# hier bestaat column label nog in X 
print(X['label'])  

ID
TCGA-02-0006    GBM
TCGA-02-0009    GBM
TCGA-02-0011    GBM
TCGA-02-0027    GBM
TCGA-02-0033    GBM
               ... 
TCGA-HT-8018    LGG
TCGA-HT-8111    LGG
TCGA-HT-8114    LGG
TCGA-HT-8563    LGG
TCGA-HT-A61A    LGG
Name: label, Length: 167, dtype: object


#### Drop label column from dataset X

In [9]:
X_train= X_train.drop(['label'], axis=1)
X_test= X_test.drop(['label'], axis=1)
X = X.drop(['label'], axis=1)

#### Oud stuk code

In [13]:
## Dit kan volgensmij beide weg:
#OUDE CODE test train split:
# data_features = pd.DataFrame(data=data_brats)
# data_train, data_test = train_test_split(data_features, test_size=0.45) # Nog bepalen wat test_size wordt

#OUDE code splitten X en Y
#  print(y)
# y = data_fill.pop('label')
# print(y)
# X = data_fill

#### No None
Deleting columns with NaN or filling them.
- Bepalen waar threshold ligt

In [10]:
# Insight in the data
print(f'OVERZICHT: {X_train.isnull().sum()}')

# If the total number of NaN observations in a column are greater than 40%, delete the entire column.
perc = 40.0
min_count = int(((100-perc)/100)*X_train.shape[0] + 1)
data_dropcolumn = X_train.dropna(axis=1, thresh=min_count)
#print(data_dropcolumn)
#print(data_dropcolumn.size)

# fill the NaN observations.
data_fill = data_dropcolumn.fillna(data_dropcolumn.median()) #Bekijken mean of median
#print(data_fill)
#print(data_fill.size)

# Inzicht in data
print(f'OVERZICHT NONONE: {data_fill.isnull().sum()}')

OVERZICHT: VOLUME_ET        0
VOLUME_NET       0
VOLUME_ED        0
VOLUME_TC        0
VOLUME_WT        0
              ... 
TGM_T_5        130
TGM_Cog_X_6    132
TGM_Cog_Y_6    132
TGM_Cog_Z_6    132
TGM_T_6        132
Length: 724, dtype: int64


C:\Users\maaik\AppData\Local\Temp\ipykernel_22232\3414857135.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data_fill = data_dropcolumn.fillna(data_dropcolumn.median()) #Bekijken mean of median


OVERZICHT NONONE: VOLUME_ET      0
VOLUME_NET     0
VOLUME_ED      0
VOLUME_TC      0
VOLUME_WT      0
              ..
TGM_dw         0
TGM_Cog_X_1    0
TGM_Cog_Y_1    0
TGM_Cog_Z_1    0
TGM_T_1        0
Length: 704, dtype: int64


#### Scale features
- Checken of Robust Scaler juiste keuze is
--> error door div/0 error!

In [12]:
# # standard scaler
# scaler = StandardScaler()
# scaler.fit(X)
# X_scaled_one = scaler.transform(X)
# print(X_scaled_one)

# # minmax scaler
# scaler_two = MinMaxScaler()
# scaler_two.fit(X)
# X_scaled_two = scaler_two.transform(X)
# print(X_scaled_two)

# # robustscaler
# scaler_three = RobustScaler()
# scaler_three.fit(X)
# X_scaled = scaler_three.transform(X)
# print(X_scaled)

#### Transform features
- We denken alleen PCA te gebruiken. Klopt het dat je dan niet ook selectie gebruikt?
- PCA gaat uit van lineair model. Hoe kunnen we weten of ons onze data daar geschikt voor is?
- Is het de bedoeling dat we ons hier verder in verdiepen of valt dat buiten de scope van het vak?
- Uitzoeken hoe we de X_test en y_test correct gereed krijgen voor PCA.

In [ ]:
'''
# Perform a PCA
pca = decomposition.PCA(n_components=2)
pca.fit(X) 
X_train_pca = pca.transform(X)

X_test_pca = pca.transform(X_test)

'''

'\n# Perform a PCA\npca = decomposition.PCA(n_components=2)\npca.fit(X) \nX_train_pca = pca.transform(X)\n\nX_test_pca = pca.transform(X_test)\n\n'

#### Classifier: kNN

In [ ]:
'''
# Fit kNN
knn = neighbors.KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_pca, y_train)
score_train = knn.score(X_train_pca, y_train)
score_test = knn.score(X_test_pca, y_test)

# Print result
print(f"Training result: {score_train}")
print(f"Test result: {score_test}")
'''